In [3]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
train.head()

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67


In [5]:
test.head()

,id,product_uid,product_title,search_term
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able
3,6,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong ties
4,7,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong tie hcc668


In [6]:
train.describe()

,id,product_uid,relevance
count,74067.000000,74067.000000,74067.000000
mean,112385.709223,142331.911553,2.381634
std,64016.573650,30770.774864,0.533984
min,2.000000,100001.000000,1.000000
25%,57163.500000,115128.500000,2.000000
50%,113228.000000,137334.000000,2.330000
75%,168275.500000,166883.500000,3.000000
max,221473.000000,206650.000000,3.000000


In [7]:
test.describe()

,id,product_uid
count,166693.000000,166693.000000
mean,123932.839741,149713.170445
std,71518.389174,37270.476881
min,1.000000,100001.000000
25%,61669.000000,116665.000000
50%,124004.000000,142445.000000
75%,187036.000000,179444.000000
max,240760.000000,224428.000000


In [10]:
data_all = pd.concat([train, test])

In [27]:
h = data_all.sort_values('id').groupby('product_uid').apply(lambda df: len(df)).hist

In [28]:
h?

In [29]:
data_all.sort_values('id').groupby('product_uid').apply(lambda df: len(df)).hist(bins=100)

In [30]:
from matplotlib import pyplot as plt
plt.show()

In [15]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')

df_train = pd.read_csv('./train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('./test.csv', encoding="ISO-8859-1")
# df_attr = pd.read_csv('../input/attributes.csv')
df_pro_desc = pd.read_csv('./product_descriptions.csv')

#df_train = df_train.head(1000)
#df_test = df_test.head(1000)
#df_pro_desc = df_pro_desc.head(1000)

num_train = df_train.shape[0]

def str_stemmer(s):
	return " ".join([stemmer.stem(word) for word in s.lower().split()])

def str_common_word(str1, str2):
	return sum(int(str2.find(word)>=0) for word in str1.split())


df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')

df_all['search_term'] = df_all['search_term'].map(lambda x:str_stemmer(x), n_jobs=4)
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stemmer(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stemmer(x))

df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)

df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title']+"\t"+df_all['product_description']

df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))

df_all = df_all.drop(['search_term','product_title','product_description','product_info'],axis=1)

df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']

y_train = df_train['relevance'].values
X_train = df_train.drop(['id','relevance'],axis=1).values
X_test = df_test.drop(['id','relevance'],axis=1).values

rf = RandomForestRegressor(n_estimators=15, max_depth=6, random_state=0, n_jobs=4)
clf = BaggingRegressor(rf, n_estimators=45, max_samples=0.1, random_state=25)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission.csv',index=False)


TypeError: map() got an unexpected keyword argument 'n_jobs'

In [13]:
RandomForestRegressor.fit?

In [9]:
df_all.head()

,id,product_uid,relevance,len_of_query,word_in_title,word_in_description
0,2,100001,3.00,2,1,1
1,3,100001,2.50,2,1,1
2,9,100002,3.00,2,1,1
3,16,100005,2.33,3,1,1
4,17,100005,2.67,3,3,2


In [64]:
train = pd.read_csv('train.csv', encoding='ISO-8859-1').head(1000)
test = pd.read_csv('test.csv', encoding='ISO-8859-1').head(1000)
product_descriptions = pd.read_csv('product_descriptions.csv', encoding='ISO-8859-1').head(1000)

df_all = pd.concat([train, test], axis=0)

df_all = pd.merge(df_all, product_descriptions, how='left', on='product_uid')
df_all.search_term = df_all.search_term.map(str_stemmer)
df_all.product_title = df_all.product_title.map(str_stemmer)
df_all.product_description = df_all.product_description.map(str_stemmer)

df_all = df_all.apply(f, axis=1)
df_all = df_all.drop(['product_title', 'product_description', 'search_term'], axis=1)


In [57]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

def str_stemmer(s):
    return " ".join([stemmer.stem(i) for i in s.split()])

def f(s):
    title = [i.lower() for i in s['product_title'].split()]
    desc = [i.lower() for i in s['product_description'].split()]
    search = [i.lower() for i in s['search_term'].split()]
    s['title_search_common'] = len(set(title) & set(search)) 
    s['desc_search_common'] = len(set(desc) & set(search)) 
    return s


In [66]:
df_all.sort_values('relevance', ascending=False)

,id,product_uid,relevance,title_search_common,desc_search_common
0,2,100001,3.0,1,1
629,2013,100358,3.0,1,2
527,1704,100299,3.0,1,1
526,1703,100299,3.0,0,1
525,1702,100299,3.0,0,1
521,1681,100295,3.0,0,0
514,1669,100293,3.0,1,1
510,1657,100289,3.0,2,2
502,1643,100286,3.0,3,3
489,1595,100280,3.0,2,1


In [14]:
df_all['common'] = 1

In [15]:
df_all

,id,product_title,product_uid,relevance,search_term,common
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angle bracket,1
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket,1
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over,1
3,16,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head,1
4,17,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower only faucet,1
5,18,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,3.00,convection otr,1
6,20,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,2.67,microwave over stove,1
7,21,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,3.00,microwaves,1
8,23,Lithonia Lighting Quantum 2-Light Black LED Em...,100007,2.67,emergency light,1
9,27,House of Fara 3/4 in. x 3 in. x 8 ft. MDF Flut...,100009,3.00,mdf 3/4,1


In [38]:
df = pd.DataFrame({'a':[1, 2, 3], 'b': [4,5,6], 'c': [7,8,9]})

In [48]:
def f(*args, **kwargs):
    print('args=|%s|' % args[0])
    return args[0]
    

In [39]:
df

,a,b,c
0,1,4,7
1,2,5,8
2,3,6,9


In [49]:
df.apply(f, axis=0)

args=|0    1
1    2
2    3
Name: a, dtype: int64|
args=|0    1
1    2
2    3
Name: a, dtype: int64|
args=|0    4
1    5
2    6
Name: b, dtype: int64|
args=|0    7
1    8
2    9
Name: c, dtype: int64|


,a,b,c
0,1,4,7
1,2,5,8
2,3,6,9


In [50]:
df.apply?

In [20]:
df_all

,id,product_title,product_uid,relevance,search_term
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angle bracket
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over
3,16,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head
4,17,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower only faucet
5,18,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,3.00,convection otr
6,20,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,2.67,microwave over stove
7,21,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,3.00,microwaves
8,23,Lithonia Lighting Quantum 2-Light Black LED Em...,100007,2.67,emergency light
9,27,House of Fara 3/4 in. x 3 in. x 8 ft. MDF Flut...,100009,3.00,mdf 3/4
